In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from pathlib import Path
import os.path
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [7]:
image_path= Path('20-50/20-50')

In [8]:
filepaths = pd.Series(list(image_path.glob(r'**/*.jpg')), name='Filepath').astype(str)
ages = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Age').astype(np.int)

images = pd.concat([filepaths, ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

/tmp/ipykernel_805/2764185558.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ages = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Age').astype(np.int)


In [9]:
train_df, test_df = train_test_split(images, test_size=0.2, shuffle=True, random_state=1)

In [10]:
train_df

,Filepath,Age
28855,20-50/20-50/train/47/150836.jpg,47
11912,20-50/20-50/train/29/147441.jpg,29
2666,20-50/20-50/train/47/174298.jpg,47
36862,20-50/20-50/train/39/149598.jpg,39
4077,20-50/20-50/train/34/156051.jpg,34
...,...,...
7813,20-50/20-50/test/44/42753.jpg,44
32511,20-50/20-50/train/26/164073.jpg,26
5192,20-50/20-50/train/40/161263.jpg,40
12172,20-50/20-50/train/22/113418.jpg,22


In [11]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [12]:
X_train=train_gen.flow_from_dataframe(
dataframe=train_df,
x_col="Filepath",
y_col="Age",
target_size=(120,120),
color_mode="rgb",
class_mode="raw",
batch_size=32,
shuffle=True,
seed=42,
subset="training")

Found 25882 validated image filenames.


In [13]:
X_test=test_gen.flow_from_dataframe(
dataframe=test_df,
x_col="Filepath",
y_col="Age",
target_size=(120,120),
color_mode="rgb",
class_mode="raw",
batch_size=32,
shuffle=False)

Found 8088 validated image filenames.


In [14]:
val_images = train_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

Found 6470 validated image filenames.


In [15]:
input=layers.Input(shape=(120,120,3))

x=layers.Conv2D(16,3,activation="relu")(input)
x=layers.BatchNormalization()(x)
x=layers.MaxPooling2D((2,2))(x)

x=layers.Conv2D(32,3,activation="relu")(x)
x=layers.BatchNormalization()(x)
x=layers.MaxPooling2D((2,2))(x)

x=layers.Conv2D(64,3,activation="relu")(x)
x=layers.BatchNormalization()(x)
x=layers.MaxPooling2D((2,2))(x)

x=layers.Flatten()(x)
x=layers.Dense(128,activation="relu")(x)
x=layers.Dropout(0.3)(x)
x=layers.Dense(64,activation="relu")(x)
x=layers.Dropout(0.3)(x)

output=layers.Dense(1,activation="linear")(x)

model=Model(input,output)
model.summary()
                             

2023-03-28 00:50:32.811939: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 00:50:32.818095: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 118, 118, 16)      448       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 16)     64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 32)       128   

In [16]:
model.compile(optimizer="adam",loss="mae")

In [17]:
#save_model
from tensorflow.keras.models import model_from_json
def save_model(model, filename= 'model-dl') :
    #serialize model to JSON
    model_json = model.to_json()
    with open(filename+".json", "w") as json_file:
        json_file.write(model_json)
    #serialize weights to HDF5
    model.save_weights(filename+'.h5')
    print("Save model to dick")\
    #create_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
# def create_model(): 
#     model = Sequential ()
#     model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(8,8,1)))
#     model.add(Flatten())
#     model.add(Dense(10, activation ='softmax'))
#     model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
#     return model

In [20]:
#train_model
# model_save= create_model()
model.fit(X_train,validation_data=val_images,epochs=50)
save_model(model, filename= 'mohinh_dudoantuoi')

Epoch 1/50
809/809 [==============================] - 282s 347ms/step - loss: 10.0693 - val_loss: 8.9225
Epoch 2/50
809/809 [==============================] - 303s 374ms/step - loss: 9.0162 - val_loss: 9.1375
Epoch 3/50
809/809 [==============================] - 320s 396ms/step - loss: 8.7228 - val_loss: 10.9352
Epoch 4/50
809/809 [==============================] - 268s 331ms/step - loss: 8.5327 - val_loss: 7.5347
Epoch 5/50
809/809 [==============================] - 291s 360ms/step - loss: 8.2750 - val_loss: 10.1029
Epoch 6/50
809/809 [==============================] - 277s 343ms/step - loss: 8.1165 - val_loss: 7.4774
Epoch 7/50
809/809 [==============================] - 276s 341ms/step - loss: 7.9477 - val_loss: 9.2263
Epoch 8/50
809/809 [==============================] - 453s 560ms/step - loss: 7.8751 - val_loss: 11.1759
Epoch 9/50
809/809 [==============================] - 298s 367ms/step - loss: 7.6944 - val_loss: 8.0533
Epoch 10/50
809/809 [==============================] - 300s 

In [13]:
from PIL import Image, ImageOps
import numpy as np
def process_and_predict(file):
    im = Image.open(file)
    width, height = im.size
    if width == height:
        im = im.resize((120,120), Image.ANTIALIAS)
    else:
        if width > height:
            left = width/2 - height/2
            right = width/2 + height/2
            top = 0
            bottom = height
            im = im.crop((left,top,right,bottom))
            im = im.resize((120,120), Image.ANTIALIAS)
        else:
            left = 0
            right = width
            top = 0
            bottom = width
            im = im.crop((left,top,right,bottom))
            im = im.resize((120,120), Image.ANTIALIAS)
            
    ar = np.asarray(im)
    ar = ar.astype('float32')
    ar /= 255.0
    ar = ar.reshape(-1, 120, 120, 3)
    
    import matplotlib.pyplot as plt
    files_to_load = 'mohinh_dudoantuoi'
    model_image = load_model(files_to_load)

    predictions = model_image.predict(ar)
    print('Ket qua du doan - hinh cua ai do:', predictions[0])

    
#     age = x.predict(ar)
        
#     print('Age:', int(age))
#     return im.resize((300,300), Image.ANTIALIAS)

In [14]:
# Dinh nghia ham/ thu tuc ho tro
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json
def load_model(filename="model-dl"):
    model = model_from_json(open(filename+'.json').read())
    model.load_weights(filename+'.h5')
    model.compile(loss="categorical_crossentropy", optimizer='adam')
    return model

In [28]:
process_and_predict('hinhdudoan/Tet.jpg')

/tmp/ipykernel_285/3655027125.py:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize((120,120), Image.ANTIALIAS)


1/1 [==============================] - 0s 93ms/step
Ket qua du doan - hinh cua ai do: [33.058113]


In [30]:
def nhan_biet_tuoi(images, image_cuaban, classid):
    j = 0
    flag = 1
    fig = plt.figure(figsize= (24,12))
    for i in range(len(targets)):
        if faces.target[i] == classid:
            if j==0:
                img_grid = fig.add_subplot(2, 5, j+1)
                print('Tuoi cua ban ')
                img_grid.imshow(image_cuaban)
            else:
                img_grid = fig.add_subplot(2,5,j+1)
                print('classid,', classid, 'sample_id', i)
                img_grid.imshow(images[i])
            j=j+1
            flag=0
        if flag:
            print('Khong co anh nao')

In [31]:
from tensorflow.keras.preprocessing.image import load_img
anh= '145207.jpg'
image_cuaban= load_img(anh, target_size=(64,64))
nhan_biet_tuoi(images, image_cuaban, classid=5)
plt.show

NameError: name 'plt' is not defined